In [2]:
# from gensim.test.utils import common_texts, get_tmpfile
# from gensim.models import Word2Vec, KeyedVectors
import numpy as np
from collections import defaultdict

In [3]:
import pandas as pd 
data = pd.read_csv('/data/Data/yntn/data_labeled.csv')
sentences = list(data["text"] )
corpus = []
for sentence in sentences:
    corpus.append([word.lower() for word in str(sentence).split()])

In [3]:
corpus

[['ngon', 'k', 'c'],
 ['sao', 'e', 'lại', 'nói', 'nv...bộ', 'c', 'hung', 'dữ', 'lắm', 'hả'],
 ['ok', 'n', 'cho', 'cái', 'hẹn', 'đi'],
 ['thùy', 'mị', 'quá', 'bạn', 'ah'],
 ['có', 'đâu', 'xé,', 'ib', 'đi'],
 ['sao', 'cũg', 'đc', 'hết'],
 ['e', 'muốn', 'ở', 'tn', 'mà'],
 ['e', 'cũng', 'ngủ', 'ngon', 'nhé'],
 ['a', 'ở', 'lại', 'đi'],
 ['lâu', 'không', 'gặp', 'nhìn', 'đẹp', 'zai', 'hẳn', 'nha'],
 ['hay', 'quá', 'p', 'oi..!'],
 ['đẹp', 'quá', 'e', 'ơi'],
 ['thương'],
 ['e', 'chưa', 'hiểu', 'lắm'],
 ['tâm', 'trạng', 'thế', 'a', 'ơi.'],
 ['buồn', 'quá', 'buồn'],
 ['ai', 'cho', 'đi', 'thả', 'thính', 'dạo', 'vậy', 'hả'],
 ['hi', 'dạ', 'e'],
 ['hic...', 'a', 'xl', 'ty', 'của', 'a...', 'a', 'xl', 'e...'],
 ['béo'],
 ['có', 'sống', 'tốt', 'cũng', 'chẳng', 'còn', 'để', 'đền', 'đáp'],
 ['àh', 'mà', 'vui', 'vẻ', 'th', 'nha', 'bữa', 'nào', 'quẩy', 'ak'],
 ['e', 'yêu', 'xưn', 'quá'],
 ['đà', 'lạt', 'chứ', 'đâu'],
 ['lâu', 'lm', 'ak...chờ', 'e', 'nổi', 'hk'],
 ['giá', 'như', 'là', 'tay', 'mh', 'th

In [7]:
class word2vec():
    def __init__(self):
        self.n = setting['n']
        self.learning_rate = setting['learning_rate']
        self.epochs = setting['epochs']
        self.window_size = setting['window_size']
        
    def word2onehot(self, word):
        word_vec = [ 0 for i in range(0, self.v_count)] # set all 0
        word_vec[self.word_index[word]] = 1 # set one hot
        return word_vec
    
    def gen_training_data(self, setting, corpus):
        word_count = defaultdict(int) # defaultdict will have a default value if that key has not been set yet
        
        for sentence in corpus:
            for word in sentence:
                word_count[word] += 1
        
        self.v_count = len(word_count.keys()) # vocab size

        self.words_list = list(word_count.keys()) # vocab
        
        self.word_index = dict((word, i) for i , word in enumerate(self.words_list)) #indexing for word in vocab
        self.index_word = dict((i, word) for i , word in enumerate(self.words_list))
        
        traning_data = [] # list of pair [onehot vector of word, list onehot vector of neighbor
        
        for sentence in corpus:
            sentence_len = len(sentence)
            
            for i, word in enumerate(sentence):
                w_target = self.word2onehot(sentence[i]) # one hot vector
                w_context = [] # list of one hot vector of neighbors
                
                for j in range(i - self.window_size, i + self.window_size + 1): #constraint: in window size
                    if ((j != i) and (j <= sentence_len - 1) and (j >= 0)): #constraint: in sentence
                        w_context.append(self.word2onehot(sentence[j])) 
                traning_data.append([w_target, w_context])
                
        return np.array(traning_data)    
    
    def forward_pass(self, w_t):
        h = np.dot(self.weight1.T, w_t)
        u = np.dot(self.weight2.T, h)
        tmp = np.exp(u - max(u))
        y_c = (tmp/tmp.sum(axis = 0)) # softmax
        return y_c, h, u
    
    def backprop(self, err, h, w_target):
        delta_weight2 = np.outer(h, err)
        delta_weight1 = np.outer(w_target,np.dot(self.weight2, err.T))
        print (err.T)
        self.weight1 = self.weight1 - (self.learning_rate * delta_weight1) #update
        self.weight2 = self.weight2 - (self.learning_rate * delta_weight2) #update
   
    def train(self, traning_data):
        
        self.weight1 = np.random.uniform(-1, 1, (self.v_count, self.n))
        self.weight2 = np.random.uniform(-1, 1, (self.n, self.v_count))
            
        for i in range (self.epochs):
            self.loss = 0
            for w_target, w_context in traning_data:
                y_predict, h, u = self.forward_pass(w_target) #vector for prediction y_predict, hidden layer h and out  put layer u
                
                err = np.sum([(y_predict - word) for word in w_context], axis = 0) # sum up the difference between y_pred and each of the context words in w_c
       
                self.backprop(err, h, w_target)
        
                self.loss += - np.sum([u[word.index(1)] for word in w_context]) + len(w_context) * np.log(np.sum(np.exp(u)))
                
            print("Epoch: " , i, "Loss: ", self.loss)  
         
    def wordtovec(self, word):
#         w_index = self.word_index[word]
#         v_w = self.w1[w_index]
       return self.weight1[self.word_index[word]]

In [5]:
word_count = defaultdict(int)
word_count

defaultdict(int, {})

In [6]:
vec = np.array([1,2,3,4,5,6,7,8,9,8,7,6,5,4,3,2,1])
np.exp(vec)

array([2.71828183e+00, 7.38905610e+00, 2.00855369e+01, 5.45981500e+01,
       1.48413159e+02, 4.03428793e+02, 1.09663316e+03, 2.98095799e+03,
       8.10308393e+03, 2.98095799e+03, 1.09663316e+03, 4.03428793e+02,
       1.48413159e+02, 5.45981500e+01, 2.00855369e+01, 7.38905610e+00,
       2.71828183e+00])

In [6]:
vec = np.array([1,2,3,4,5,6,7,8,9,8,7,6,5,4,3,2,1])
tmp = np.exp(vec - max(vec))
y_c = (tmp/tmp.sum(axis = 0))
# tmp
y_c

array([1.55051013e-04, 4.21472352e-04, 1.14568064e-03, 3.11428285e-03,
       8.46549849e-03, 2.30116107e-02, 6.25520432e-02, 1.70034082e-01,
       4.62200557e-01, 1.70034082e-01, 6.25520432e-02, 2.30116107e-02,
       8.46549849e-03, 3.11428285e-03, 1.14568064e-03, 4.21472352e-04,
       1.55051013e-04])

In [ ]:
setting = {'n': 10, 'learning_rate': 0.01, 'epochs':50, 'window_size': 3}
# corpus = [ ['trog', 'ảnh', 'thế', 'thôi', 'chứ', 'ngoài', 'còn', 'bé', 'mà']]
w2v = word2vec()

training_data = w2v.gen_training_data(setting, corpus)

w2v.train(training_data)